In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import decimal

In [3]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext

In [4]:
spark = SparkSession \
    .builder \
    .appName("PMPM PROCESSING") \
    .master("local") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.executor.cores", 15) \
    .config("spark.dynamicAllocation.minExecutors", 1) \
    .config("spark.dynamicAllocation.maxExecutors", 1) \
    .config('spark.executor.memory', '45g') \
    .config('spark.driver.memory', '45g') \
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
    .config('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true') \
    .getOrCreate()

In [5]:
path = '../../data/PMPM/'
#sc = SQLContext(context)
pmpm = spark.read.parquet(path)

#Add Allow Ip and Allow ER(True to add the columns in the final dataset)
add_ip_er = True

### Get Only Eligible Patients

In [6]:
pmpm = pmpm.filter(pmpm.IS_CANARY_ELIGIBLE == 1)

### Decide which columns to keep and to drop

In [7]:
import re
schema = pmpm.schema.names
match = re.compile(r'^PERS_ID|^CC[\w\s]|^ALLOW_AMT\W*|MYR|AGE_AT_MIDMONTH|MBR_GNDR')

columns_to_keep = []
columns_to_drop = []
for sh in schema:
    if match.match(sh) is not None:
        if 'ALLOW_AMT_' not in sh:
            columns_to_keep.append(sh)
    else:
        columns_to_drop.append(sh)


### Keep only a subset of the columns

In [8]:
total_cols = np.intersect1d(columns_to_keep, pmpm.columns)
dr = len(columns_to_drop)
kp = len(columns_to_keep)
total = len(schema)
len(total_cols), dr, kp, dr+kp, total
total_cols = list(total_cols)

if add_ip_er:
    print('Adding IP ER')

    total_cols.insert(2, 'ALLOW_IP')
    total_cols.insert(2, 'ALLOW_ER')
    total_cols.insert(2,'MEDICAID_LOB')

Adding IP ER


### Filter from spark dataframe the relevant columns

In [9]:
pmpm_eligible = pmpm.select(total_cols)

### Write the PMPM file locally into csv

In [10]:
%%time
pmpm_eligible.write.csv('temp_pmpm.csv', header = True)


CPU times: user 26.3 ms, sys: 8.5 ms, total: 34.8 ms
Wall time: 4min 36s


### Read PMPM file from disk into pandas

In [11]:
%%time
import glob
path = r'temp_pmpm.csv' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

CPU times: user 1min 1s, sys: 4.32 s, total: 1min 5s
Wall time: 1min 5s


In [12]:
!rm -rf temp*

### Rearrange The columns of the pandas array

In [13]:
%%time
cols = list(df.columns)
col = [cols[-1]] +[cols[-2]] + [cols[-3]] + cols[0:-3]
df = df[col]
df.MYR = df.MYR.astype(int)
df.PERS_ID = df.PERS_ID.astype(str)
df.AGE_AT_MIDMONTH = df.AGE_AT_MIDMONTH.astype(int)
df.ALLOW_AMT = df.ALLOW_AMT.astype(int)

CPU times: user 13.5 s, sys: 6.49 s, total: 20 s
Wall time: 20 s


In [14]:
df = df.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)

In [15]:
df.shape

(9650961, 77)

### Save pandas File locally as pandas Array

In [16]:
%%time
from pathlib import Path
Path('../../temp').mkdir(parents=True, exist_ok=True)
df.to_pickle('../../temp/PMPM.p')

CPU times: user 6.67 s, sys: 8.67 s, total: 15.3 s
Wall time: 15.3 s


### Save to S3

In [17]:
%%time
import boto3
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'

if add_ip_er:
    s3_name = 'ELIGIBLE_PLUS_AUGUST2021.p'
    print('Added Ip ER, LOB')
else:
    s3_name = 'ELIGIBLE'

s3.upload_file( '../../temp/PMPM.p',
                bucket, 
                'rpi/DATA_GEORGE/PMPM_PICKLE/'+s3_name,
                ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})

Added Ip ER, LOB
CPU times: user 27.8 s, sys: 10.1 s, total: 37.9 s
Wall time: 2min 50s
